[View in Colaboratory](https://colab.research.google.com/github/DongminWu/MLpractice/blob/master/test_deep_siamese_chinese_company_names.ipynb)

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


from scipy.stats import norm

from scipy.signal import triang

from sklearn.utils import shuffle


# Data Generating

In [0]:
num_of_each_type = 40
num_of_weekend = 13
day_length = 1440

test_prop = 0.2


In [0]:
!rm *.txt

In [106]:
from google.colab import files
uploaded = files.upload()

Saving company.txt to company.txt


In [0]:
data = pd.read_csv("company.txt", header=None)[750:1000]
# data = pd.read_csv("company_short.txt", header=None)[:250]

In [0]:
data = shuffle(data)

In [157]:
all_words = []
all_samples = []
max_len = 0

for it in data.iloc[:].iterrows():
  item = it[1][0]
  list_item = list(item)
  all_samples.append(list_item)
  max_len = len(list_item) if max_len < len(list_item) else max_len
  for e in list_item:
    all_words.append(e)

all_words = np.array(all_words)
uni_words = np.unique(all_words)

print('words:', uni_words)
print('max length:', max_len)


one_hot = np.eye(uni_words.shape[0])

word_dict = {}
for i, e in enumerate(uni_words):
  word_dict[e] = i


words: ['(' ')' '《' '》' '一' '万' '三' '上' '下' '世' '业' '东' '丝' '中' '丰' '乐' '九' '书'
 '事' '二' '云' '交' '产' '人' '仁' '仑' '仓' '代' '仪' '任' '份' '企' '会' '传' '住' '佳'
 '供' '侨' '俊' '保' '信' '修' '倩' '健' '储' '光' '公' '关' '兴' '具' '农' '冠' '冶' '出'
 '分' '划' '列' '创' '制' '刷' '券' '力' '办' '加' '务' '动' '劳' '勘' '包' '化' '北' '医'
 '升' '华' '南' '博' '卢' '印' '原' '厦' '县' '友' '双' '发' '变' '口' '司' '合' '同' '向'
 '吴' '告' '员' '和' '咨' '品' '售' '商' '啤' '啸' '器' '团' '园' '国' '图' '土' '地' '场'
 '城' '培' '基' '塑' '境' '处' '备' '复' '夏' '外' '大' '天' '套' '威' '媒' '子' '字' '学'
 '安' '宗' '宝' '实' '客' '宾' '寿' '小' '就' '局' '屋' '展' '属' '山' '岛' '岩' '峰' '川'
 '工' '巴' '市' '师' '平' '广' '应' '店' '康' '建' '开' '弋' '强' '影' '待' '微' '德' '心'
 '志' '思' '总' '恒' '息' '戎' '成' '房' '所' '托' '技' '投' '报' '拓' '招' '振' '探' '控'
 '摄' '放' '政' '数' '料' '斯' '新' '方' '施' '旅' '旦' '时' '昂' '昌' '明' '易' '星' '昭'
 '智' '月' '有' '服' '木' '本' '术' '机' '杂' '材' '来' '杰' '松' '构' '林' '桥' '梅' '械'
 '检' '森' '楼' '模' '欣' '欧' '民' '气' '水' '永' '汇' '江' '汽' '沪' '油' '泥' '泰' '泽'
 '泾' '洋' '流' '测' '济' '浚' '浦' '海' '消' '润' '深'

In [158]:
samples = []
for item in all_samples:
  one_line = []
  for e in item:
    one_line.append(word_dict[e])
  samples.append(np.array(one_line))

  
embedded_words = np.zeros([len(data), max_len, len(uni_words)])
for i, item in enumerate(all_samples):
  for j, character in enumerate(item):
    embedded_words[i, j, word_dict[character]] = 1
print("embedded data:", embedded_words.shape)


embedded data: (250, 18, 433)


In [159]:
test_len = int(data.shape[0] * test_prop)

test_data = embedded_words[:test_len,:,:]
train_data = embedded_words[test_len:,:,:]

print("test_data shape", test_data.shape)
print("train_data shape", train_data.shape)

test_data shape (50, 18, 433)
train_data shape (200, 18, 433)


## Train set


Train set will consist two parts:

1. Same name -> target = 1
2. different name -> target = 0

```

for e in train_data
    same = (e,e)
    different = (e, train_data\e)
```

In [160]:
def pre(data):
  data_idx = list(range(len(data)))

  same_l_idx = data_idx
  same_r_idx = data_idx
  same_target = np.ones(len(data))

  diff_l_idx = shuffle(data_idx)
  diff_r_idx = shuffle(data_idx)
  diff_target = np.zeros(len(data))


  for i, pair in enumerate(zip(diff_l_idx, diff_r_idx)):
    if pair[0] == pair[1]:
      diff_target[i] = 1
  
  ret_l = same_l_idx + diff_l_idx
  ret_r = same_r_idx + diff_r_idx
  ret_target = np.concatenate([same_target , diff_target])
  ret_l, ret_r, ret_target = shuffle(ret_l, ret_r, ret_target)

  return ret_l, ret_r, ret_target

tr_l, tr_r, tr_target = pre(train_data)
te_l, te_r, te_target = pre(test_data)


print('left train data length:', len(tr_l))
print('left test data length:', len(te_l))



left train data length: 400
left test data length: 100


# model building


## 1D-CNN

In [0]:


dim_dense = 32

dropout_rate = 0.2
regularizer_coef = 0.01
rolling_window = 60




In [0]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Input, Dense, Lambda
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import Model
from keras import optimizers

In [163]:


input_l = Input(shape=(max_len, len(uni_words)), name = 'input_layer_l')
input_r = Input(shape=(max_len, len(uni_words)), name = 'input_layer_r')

conv_1_l = Conv1D(256, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_1_l")(input_l)
conv_1_r = Conv1D(256, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_1_r")(input_r)

pool_1_l = MaxPooling1D(pool_size=2, name="pool_1_l")(conv_1_l)
pool_1_r= MaxPooling1D(pool_size=2, name="pool_1_r")(conv_1_r)

drop_1_l = Dropout(dropout_rate)(pool_1_l)
drop_1_r = Dropout(dropout_rate)(pool_1_r)


conv_2_l = Conv1D(128, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_3_l")(drop_1_l)
conv_2_r = Conv1D(128, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_3_r")(drop_1_r)

pool_2_l = MaxPooling1D(pool_size=2, name="pool_4_l")(conv_2_l)
pool_2_r= MaxPooling1D(pool_size=2, name="pool_4_r")(conv_2_r)

drop_2_l = Dropout(dropout_rate)(pool_2_l)
drop_2_r = Dropout(dropout_rate)(pool_2_r)

# conv_3_l = Conv1D(32, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_5_l")(drop_2_l)
# conv_3_r = Conv1D(32, kernel_size=2, kernel_regularizer=l2(regularizer_coef), name="conv_5_r")(drop_2_r)

# pool_3_l = MaxPooling1D(pool_size=2, name="pool_6_l")(conv_3_l)
# pool_3_r= MaxPooling1D(pool_size=2, name="pool_6_r")(conv_3_r)

# drop_3_l = Dropout(dropout_rate)(pool_3_l)
# drop_3_r = Dropout(dropout_rate)(pool_3_r)


flat_l = Flatten(name="flat_l")(drop_2_l)
flat_r = Flatten(name="flat_r")(drop_2_r)

dense_7_l = Dense(dim_dense, name="dense_7_l")(flat_l)
dense_7_r = Dense(dim_dense, name="dense_7_r")(flat_r)



L1_distance = lambda x: K.abs(x[0]-x[1])

both = Lambda(L1_distance, output_shape=lambda x: x[0])([dense_7_l, dense_7_r])
# both = merge([flat1, flat2], mode=L1_distance, output_shape=lambda x: x[0])

prediction = Dense(1,activation='sigmoid', name="output")(both)


model = Model([input_l, input_r], prediction)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_l (InputLayer)      (None, 18, 433)      0                                            
__________________________________________________________________________________________________
input_layer_r (InputLayer)      (None, 18, 433)      0                                            
__________________________________________________________________________________________________
conv_1_l (Conv1D)               (None, 17, 256)      221952      input_layer_l[0][0]              
__________________________________________________________________________________________________
conv_1_r (Conv1D)               (None, 17, 256)      221952      input_layer_r[0][0]              
__________________________________________________________________________________________________
pool_1_l (

In [0]:
# for i in range(30):
#   print("=========> epoch:", i)


batch_size = 32
num_epoches = 30

val_prop = 0.8

learning_rate = 0.005

val_pos = int(len(tr_l) * val_prop)

ttr_l = tr_l[val_pos:]
ttr_r = tr_r[val_pos:]
ttr_target = tr_target[val_pos:]

val_l = tr_l[:val_pos]
val_r = tr_r[:val_pos]
val_target = tr_target[:val_pos]


In [0]:
model.compile(optimizers.Adam(learning_rate), loss="binary_crossentropy", metrics=['acc'])

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]
model.reset_states()

In [145]:
!ps -aux

USER       PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root         1  0.0  0.0  39144  5152 ?        Ss   07:39   0:00 /bin/bash -e /d
root        69  0.0  0.2 678296 29408 ?        Sl   07:40   0:05 node /tools/nod
root        79  0.2  0.3 723424 50596 ?        Sl   07:40   0:46 /tools/node/bin
root        89  0.2  0.4 213128 60752 ?        Sl   07:40   0:41 /usr/bin/python
root       435 43.2 51.8 46249004 6911308 ?    Ssl  12:08  17:36 /usr/bin/python
root      1246  113  0.0  33960  4772 pts/0    Ss+  12:49   0:01 /bin/sh -c ps -
root      1247  0.0  0.0  63304  6628 pts/0    R+   12:49   0:00 ps -aux


In [166]:
model.fit([train_data[tr_l],train_data[tr_r]], tr_target, epochs= 100, validation_split=0.2)

Train on 320 samples, validate on 80 samples
Epoch 1/100
320/320 [==============================] - 3s 9ms/step - loss: 5.8960 - acc: 0.4719 - val_loss: 2.1663 - val_acc: 0.4125
Epoch 2/100
320/320 [==============================] - 0s 658us/step - loss: 1.5161 - acc: 0.4688 - val_loss: 1.2022 - val_acc: 0.4750
Epoch 3/100
320/320 [==============================] - 0s 665us/step - loss: 1.1023 - acc: 0.4906 - val_loss: 0.9944 - val_acc: 0.4750
Epoch 4/100
320/320 [==============================] - 0s 648us/step - loss: 0.8841 - acc: 0.4719 - val_loss: 0.8141 - val_acc: 0.3625
Epoch 5/100
320/320 [==============================] - 0s 661us/step - loss: 0.7859 - acc: 0.5250 - val_loss: 0.7575 - val_acc: 0.4750
Epoch 6/100
320/320 [==============================] - 0s 657us/step - loss: 0.7392 - acc: 0.4906 - val_loss: 0.7274 - val_acc: 0.4750
Epoch 7/100
320/320 [==============================] - 0s 674us/step - loss: 0.7210 - acc: 0.4469 - val_loss: 0.7140 - val_acc: 0.4750
Epoch 8/100


320/320 [==============================] - 0s 655us/step - loss: 0.6972 - acc: 0.5000 - val_loss: 0.6933 - val_acc: 0.5250
Epoch 20/100
320/320 [==============================] - 0s 663us/step - loss: 0.7293 - acc: 0.4500 - val_loss: 0.7268 - val_acc: 0.4750
Epoch 21/100
320/320 [==============================] - 0s 679us/step - loss: 0.6972 - acc: 0.5094 - val_loss: 0.6931 - val_acc: 0.5250
Epoch 22/100
320/320 [==============================] - 0s 614us/step - loss: 0.6996 - acc: 0.4594 - val_loss: 0.6941 - val_acc: 0.5250
Epoch 23/100
320/320 [==============================] - 0s 648us/step - loss: 0.6965 - acc: 0.4875 - val_loss: 0.7170 - val_acc: 0.4750
Epoch 24/100
320/320 [==============================] - 0s 631us/step - loss: 0.7113 - acc: 0.4531 - val_loss: 0.6999 - val_acc: 0.4750
Epoch 25/100
320/320 [==============================] - 0s 652us/step - loss: 0.7066 - acc: 0.5125 - val_loss: 0.7005 - val_acc: 0.4750
Epoch 26/100
320/320 [==============================] - 0s 65

Epoch 36/100
320/320 [==============================] - 0s 657us/step - loss: 0.6970 - acc: 0.4688 - val_loss: 0.6939 - val_acc: 0.5250
Epoch 37/100
320/320 [==============================] - 0s 655us/step - loss: 0.7000 - acc: 0.4719 - val_loss: 0.6989 - val_acc: 0.4750
Epoch 38/100
320/320 [==============================] - 0s 660us/step - loss: 0.6988 - acc: 0.5125 - val_loss: 0.7007 - val_acc: 0.4750
Epoch 39/100
320/320 [==============================] - 0s 669us/step - loss: 0.6987 - acc: 0.4906 - val_loss: 0.7076 - val_acc: 0.4750
Epoch 40/100
320/320 [==============================] - 0s 643us/step - loss: 0.7011 - acc: 0.5000 - val_loss: 0.6977 - val_acc: 0.5250
Epoch 41/100
320/320 [==============================] - 0s 651us/step - loss: 0.7098 - acc: 0.4719 - val_loss: 0.7035 - val_acc: 0.4750
Epoch 42/100
320/320 [==============================] - 0s 655us/step - loss: 0.7058 - acc: 0.4313 - val_loss: 0.7007 - val_acc: 0.5250
Epoch 43/100
320/320 [==========================

Epoch 53/100
320/320 [==============================] - 0s 634us/step - loss: 0.7239 - acc: 0.5125 - val_loss: 0.7162 - val_acc: 0.4750
Epoch 54/100
320/320 [==============================] - 0s 644us/step - loss: 0.7170 - acc: 0.4844 - val_loss: 0.7287 - val_acc: 0.4750
Epoch 55/100
320/320 [==============================] - 0s 649us/step - loss: 0.7191 - acc: 0.5062 - val_loss: 0.7165 - val_acc: 0.4750
Epoch 56/100
320/320 [==============================] - 0s 641us/step - loss: 0.7127 - acc: 0.5125 - val_loss: 0.7071 - val_acc: 0.4750
Epoch 57/100
320/320 [==============================] - 0s 664us/step - loss: 0.7067 - acc: 0.5187 - val_loss: 0.7011 - val_acc: 0.5250
Epoch 58/100
320/320 [==============================] - 0s 643us/step - loss: 0.7070 - acc: 0.4875 - val_loss: 0.7012 - val_acc: 0.4750
Epoch 59/100
320/320 [==============================] - 0s 664us/step - loss: 0.7015 - acc: 0.4938 - val_loss: 0.7045 - val_acc: 0.4750
Epoch 60/100
320/320 [==========================

320/320 [==============================] - 0s 664us/step - loss: 0.6936 - acc: 0.5125 - val_loss: 0.6950 - val_acc: 0.4750
Epoch 71/100
320/320 [==============================] - 0s 661us/step - loss: 0.6935 - acc: 0.5125 - val_loss: 0.6957 - val_acc: 0.4750
Epoch 72/100
320/320 [==============================] - 0s 652us/step - loss: 0.6947 - acc: 0.5125 - val_loss: 0.6967 - val_acc: 0.4750
Epoch 73/100
320/320 [==============================] - 0s 668us/step - loss: 0.6951 - acc: 0.4688 - val_loss: 0.6934 - val_acc: 0.5250
Epoch 74/100
320/320 [==============================] - 0s 645us/step - loss: 0.6939 - acc: 0.4906 - val_loss: 0.6942 - val_acc: 0.4750
Epoch 75/100
320/320 [==============================] - 0s 695us/step - loss: 0.6930 - acc: 0.5187 - val_loss: 0.6949 - val_acc: 0.4750
Epoch 76/100
320/320 [==============================] - 0s 639us/step - loss: 0.6942 - acc: 0.5125 - val_loss: 0.6950 - val_acc: 0.4750
Epoch 77/100
320/320 [==============================] - 0s 64

Epoch 88/100
320/320 [==============================] - 0s 666us/step - loss: 0.6940 - acc: 0.4906 - val_loss: 0.6925 - val_acc: 0.5250
Epoch 89/100
320/320 [==============================] - 0s 652us/step - loss: 0.6942 - acc: 0.4906 - val_loss: 0.6933 - val_acc: 0.4750
Epoch 90/100
320/320 [==============================] - 0s 660us/step - loss: 0.6927 - acc: 0.5250 - val_loss: 0.6953 - val_acc: 0.4750
Epoch 91/100
320/320 [==============================] - 0s 637us/step - loss: 0.6931 - acc: 0.5125 - val_loss: 0.6967 - val_acc: 0.4750
Epoch 92/100
320/320 [==============================] - 0s 643us/step - loss: 0.6932 - acc: 0.5125 - val_loss: 0.6949 - val_acc: 0.4750
Epoch 93/100
320/320 [==============================] - 0s 632us/step - loss: 0.6933 - acc: 0.5125 - val_loss: 0.6942 - val_acc: 0.4750
Epoch 94/100
320/320 [==============================] - 0s 652us/step - loss: 0.6932 - acc: 0.5125 - val_loss: 0.6951 - val_acc: 0.4750
Epoch 95/100
320/320 [==========================

In [49]:

for n in range(num_epoches):

  print('epoch %d of %d' % (n+1, num_epoches))

  num_iteration = len(ttr_l)//batch_size
  for i in range(num_iteration):
    s = i*batch_size
    e = (i+1)*batch_size
    tr_loss = model.train_on_batch([train_data[ttr_l[s:e]],train_data[ttr_r[s:e]]], ttr_target[s:e])
  validation = model.evaluate([train_data[val_l], train_data[val_r]], val_target)
  print("  train loss:",tr_loss ,"evaluation:", validation)


epoch 1 of 30
160/160 [==============================] - 0s 2ms/step
  train loss: [2.9539633, 0.5] evaluation: [2.728298044204712, 0.4875]
epoch 2 of 30
160/160 [==============================] - 0s 205us/step
  train loss: [1.7108607, 0.53125] evaluation: [1.6610628843307496, 0.4875]
epoch 3 of 30
160/160 [==============================] - 0s 190us/step
  train loss: [1.0533506, 0.53125] evaluation: [1.0395971536636353, 0.54375]
epoch 4 of 30
160/160 [==============================] - 0s 218us/step
  train loss: [0.8822906, 0.46875] evaluation: [0.8814955115318298, 0.4875]
epoch 5 of 30
160/160 [==============================] - 0s 209us/step
  train loss: [0.83635944, 0.53125] evaluation: [0.8320917129516602, 0.4875]
epoch 6 of 30
160/160 [==============================] - 0s 237us/step
  train loss: [0.76853216, 0.53125] evaluation: [0.7733056426048279, 0.4875]
epoch 7 of 30
160/160 [==============================] - 0s 224us/step
  train loss: [0.76467794, 0.46875] evaluation: [0.

160/160 [==============================] - 0s 264us/step
  train loss: [0.71099997, 0.53125] evaluation: [0.7155989050865174, 0.4875]
epoch 23 of 30
160/160 [==============================] - 0s 223us/step
  train loss: [0.7310233, 0.53125] evaluation: [0.7353568792343139, 0.4875]
epoch 24 of 30
160/160 [==============================] - 0s 211us/step
  train loss: [0.700984, 0.53125] evaluation: [0.7037813305854798, 0.4875]
epoch 25 of 30
160/160 [==============================] - 0s 240us/step
  train loss: [0.6961221, 0.5625] evaluation: [0.6962440371513366, 0.4875]
epoch 26 of 30
160/160 [==============================] - 0s 234us/step
  train loss: [0.6960712, 0.53125] evaluation: [0.695504879951477, 0.4875]
epoch 27 of 30
160/160 [==============================] - 0s 217us/step
  train loss: [0.6944392, 0.53125] evaluation: [0.6949926376342773, 0.4875]
epoch 28 of 30
160/160 [==============================] - 0s 261us/step
  train loss: [0.6961707, 0.53125] evaluation: [0.6947717

# Conclusion

When the data set is too large, it will introduce too many dimensions, which will lead to the explotion of the error rate. The solution might be the Word2Vec
